In [1]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import csv
import time
import sqlite3

In [3]:
#get all product
driver = webdriver.Chrome('chromedriver')
driver.implicitly_wait(30)
driver.maximize_window()
driver.get('https://www.tokopedia.com/bajuanakkitaa/page/2')
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(3)

elems = driver.find_elements_by_xpath("//a")
data = []
for elem in elems:
    
    data_test = elem.get_attribute("data-testid")
    if data_test == "lnkProductContainer":
        href = elem.get_attribute("href")
        product_name = elem.text
        product_name = product_name.replace('Produk Unggulan\n','')
        product_name = product_name.replace('Stok Kosong\n','')
        products = product_name.split('\n')
        row = [href,products[0],products[1]]
        data.append(row)

driver.quit()
df = pd.DataFrame(data,columns=['href','name','price'])
# df.head(n=10)
# df.to_csv(r'toped_master_data.csv', index = False, header=True, doublequote=True)
conn = sqlite3.connect('scrap.bajuanakkita.db')
df.to_sql('BAJUANAKKITA_MASTER', conn, if_exists='append', index = False)

In [5]:
#grab detail
conn = sqlite3.connect('scrap.bajuanakkita.db')
cur = conn.cursor()
cur.execute('select a.href, a.name from BAJUANAKKITA_MASTER a left join BAJUANAKKITA_IMAGES b on a.name=b.name where b.name is null limit 10')
reader = cur.fetchall()
data_images = []
data_weight = []
data_variant = []
data_desc = []
for line in reader:
    chrome_options = webdriver.ChromeOptions()
    prefs = {"profile.default_content_setting_values.notifications" : 2}
    chrome_options.add_experimental_option("prefs",prefs)
    driver = webdriver.Chrome('chromedriver',options=chrome_options)
    driver.implicitly_wait(30)
    driver.maximize_window()
    driver.get(line[0])
    
    # fetch images
    elems = driver.find_elements_by_xpath('//div[@data-testid="PDPImageThumbnail"]')
    counting = 0
    product_image = []
    for elem in elems:
        counting = counting + 1
        elem.click()
        time.sleep(3)
        elems = driver.find_element_by_xpath('//img[@alt="product image"]')
        product_image.append(elems.get_attribute("src"))
        if counting == 5:
            break
    time.sleep(2)
    if len(product_image) == 5:
        rows = [line[1],product_image[0],product_image[1],product_image[2],product_image[3],product_image[4]]
    if len(product_image) == 4:
        rows = [line[1],product_image[0],product_image[1],product_image[2],product_image[3],None]
    if len(product_image) == 3:
        rows = [line[1],product_image[0],product_image[1],product_image[2],None, None]
    if len(product_image) == 2:
        rows = [line[1],product_image[0],product_image[1],None, None, None]
    if len(product_image) == 1:
        rows = [line[1],product_image[0],None, None, None, None]
    # print(rows)
    data_images.append(rows)
    
    #fetch variant
    elems = driver.find_elements_by_xpath("//div")
    for elem in elems:
        data_test = elem.get_attribute("data-testid")
        if data_test == "lblPDPProductVariantukuran":
            row = [line[1], elem.text]
            data_variant.append(row)
        if data_test == "lblPDPProductVariantwarna":
            row = [line[1], elem.text]
            data_variant.append(row)
    
    # fetch weight
    elems = driver.find_elements_by_xpath('//p[@data-testid="PDPDetailWeightValue"]')
    for elem in elems:
        data_test = elem.get_attribute("data-testid")
        if data_test == "PDPDetailWeightValue":
            product_weight = elem.text
            row = [line[1], product_weight]
            data_weight.append(row)
    
    #fetch description and weight
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    elems = driver.find_elements_by_xpath('//p[@data-testid="lblPDPDeskripsiProduk"]')
    for elem in elems:
        data_test = elem.get_attribute("data-testid")
        if data_test == "lblPDPDeskripsiProduk":
            product_desc = elem.text
            row = [line[1], product_desc]
            data_desc.append(row)
            
    driver.quit()
    # insert to DB
    
    # insert images
    df = pd.DataFrame(data_images,columns=['name','images1','images2','images3','images4','images5'])
    df.to_sql('BAJUANAKKITA_IMAGES', conn, if_exists='append', index = False)

    #insert weight
    df = pd.DataFrame(data_weight,columns=['name','images'])
    df.to_sql('BAJUANAKKITA_WEIGHT', conn, if_exists='append', index = False)
    
    #insert description
    df = pd.DataFrame(data_desc,columns=['name','images'])
    df.to_sql('BAJUANAKKITA_DESC', conn, if_exists='append', index = False)

    #insert variant
    df = pd.DataFrame(data_variant,columns=['name','images'])
    df.to_sql('BAJUANAKKITA_VARIANT', conn, if_exists='append', index = False)
    
    data_images.clear()
    data_weight.clear()
    data_variant.clear()
    data_desc.clear()
    time.sleep(2)
